In [1]:
import re
import numpy as np
import pandas as pd

In [4]:
### 1. 删除不符合要求的数据点
category = "val"
df = pd.read_csv("/fossfs/skguan/data_fusion/dataset_23_rpt(s)_train.csv")
df = pd.DataFrame(df[f'{category}'].dropna(), columns=[f'{category}'])
pattern = "(?P<site>s\\d+) (?P<year>\\d+)-(?P<month>\\d+)-(?P<day>\\d+)"
with open("logs/training_20250623_vit(Season)_2158554.out", 'r') as f:
    lines = f.readlines()
    for line in lines:
        if "00:00:00" in line:
            sites = np.array(list(map(lambda x: x.split("/")[-2], df[f'{category}'])))
            dates = np.array(list(map(lambda x: re.match(".*_\\d{6}_(\\d{8})*", x.split("/")[-1]).group(1),
                                      df[f'{category}'])))
            regex = re.match(pattern, line)
            site = regex.group("site")
            date = regex.group("year") + regex.group("month") + regex.group("day")
            idx = np.argwhere((sites == site) & (dates == date))
            if len(idx) == 0:
                continue
            index = int(idx.squeeze())
            index_names = df[df[f'{category}'] == df[f'{category}'].iloc[index]].index
            df.drop(index_names, inplace=True)
df.to_csv("/fossfs/skguan/data_fusion/dataset_23_rpt(s)_val.csv")